In [1]:
## AFTER each FC file is consolidated and combined with Master file,
# this script will produce a file for Actuals and one for PrevFC


import pyodbc     # connect to Data Bases using ODBC
import pandas as pd     # manipulate data in a tabular data format
import getpass  # Mask Passwords
#import csv     # Import and save CSV file extension 
from datetime import datetime, timedelta     # manipulate dates
import glob      # Retrieves the list of files matching the specified pattern
import matplotlib.pyplot as plt # Data Graphics
import time

NPD = pd.ExcelFile(r"C:\Users\andrew.schelberg\OneDrive - Vontier\Desktop\Financial Reporting\2023\0 - RevDm\00 - Trial Process\Data Files\2023_NPD_Master.xlsx")
NPD_data = NPD.parse('Sheet_1')

# Replace all NaN with 0
NPD_data = NPD_data.fillna(0)

# Create new column in df with # extracted from 'Forecast Cycle' column
forecast_nums = []
x = 0
for i in NPD_data['Forecast Cycle']:
    # appends extracted FC num as int and strips white space
    forecast_nums.append(int(NPD_data['Forecast Cycle'][x][7:].strip()))
    x += 1

# Set column with forecast number from list above    
NPD_data['Forecast Number'] = forecast_nums

# generate unique list of forecast cycle, numbers only
forecast_num_list = [*set(forecast_nums)]

# assign latest forecast submitted, for ref later
forecast_latest = max(forecast_num_list)
forecast_prev = 0
if forecast_latest > 1:
    forecast_prev = forecast_latest - 1
else:
    forecast_prev = forecast_latest

# generate unique list of month short names
months_sn = [*set(NPD_data['Month'])]

# DICTIONARIES
# for FC number key with month short name (month_sn)
fc_month_dict = {1:"Target", 2:'Jan', 3:'Feb', 4:'Mar', 5:'Apr', 6:'May', 7:'Jun',
                 8:'Jul', 9:'Aug', 10:'Sep', 11:'Oct', 12:'Nov', 13:'Dec'}

# for regions
regions_dict = {"ANZ":"AMEA","China":"AMEA","India":"AMEA","Middle East & Africa":"AMEA","South East Asia":"AMEA",
                "Latin America":"Americas","North America":"Americas", "IS360":"IS360", "EU - AutoTank":"Europe", 
                "EU - Italy":"Europe", "EU - Salzkotten":"Europe", "EU - UK":"Europe", "Europe":"Europe",
                "Veeder-Root":"Veeder-Root","Orpak - DOMS":"Orpak - DOMS", "Israel & Turkey":"Israel & Turkey",
               "AMEA":"AMEA"}


# Uses dictionary to create new column with the Region each Reporting Business falls within
regions_list = []
x = 0
for i in NPD_data['Reporting Business']:
    if i == "Orpak - DOMS" or i == "Veeder-Root" or i == "IS360":
        target_market = NPD_data['Targeted Market'][x]
        regions_list.append(regions_dict[target_market])
    else:
        regions_list.append(regions_dict[i])
    x += 1
    
NPD_data["Region"] = regions_list

# Create new df -> NPD_PrevFV
NPD_Actual = NPD_data
NPD_PrevFC = NPD_data

# Code block below is to determine which submission values to keep based on Forecast Cycle, Latest FC, and Month
fc_filter = []
x=0

for i in NPD_Actual['Forecast Number']:
    
    # assign fc as the fc num at x index (int value)
    fc = NPD_Actual['Forecast Number'][x]
    
    # assign month as month short name at x index (str value)
    month= NPD_Actual['Month'][x]
    
    # assign fcn as month number from month column as index(x) (int value)
    fcn = list(fc_month_dict.keys())[list(fc_month_dict.values()).index(month)]
    
    # appends filter value and strips white space
    
    # checks if forecast cycle is 1 (target/budget), need to be kept
    if fc == 1:
        fc_filter.append("Keep")
    
    # checks if FC number is prior to latest FC
    elif fc < forecast_latest:
        
        # check truth of fc number compared to month value (uses dictionary key's to find value)
        if fc_month_dict[fc].strip().lower() == month.strip().lower():
            fc_filter.append("Keep")
        else:
            fc_filter.append("REMOVE")
            
    # checks when FC number is the latest FC
    elif fc == forecast_latest:
    
        # check truth of fc number compared to month value (uses dictionary value to find key)
        # if the month columns value is less than (before) the forecast cycle num, "remove"
        if fcn < fc:
            fc_filter.append("REMOVE")
        else:
            fc_filter.append("Keep")
        
    else:
        fc_filter.append("Fix")
        
        
    x += 1
    
# Set column name "Value Inclusion" with resulting string from list above    
NPD_Actual['Value Inclusion'] = fc_filter

# Filter df's (df of actuals)
NPD_Actual = NPD_Actual[NPD_Actual['Value Inclusion'] == "Keep"]
NPD_Actual = NPD_Actual[NPD_Actual['Trade Revenue'] == "Vitality"] 

# Remove unnecessary columns from dataframe (these were used to clean and format the final data)
NPD_Actual = NPD_Actual.drop('Value Inclusion',axis=1)
NPD_Actual = NPD_Actual.drop('Forecast Number',axis=1)

# Code block below is to determine which submission values for previous Forecast Cycle
fc_prev_filter = []
x=0

for i in NPD_PrevFC['Forecast Number']:
    
    # assign fc as the fc num at x index (int value)
    fc = NPD_PrevFC['Forecast Number'][x]
    
    # assign month as month short name as x index (str value)
    month= NPD_PrevFC['Month'][x]
    
    # assign fcn as month number from month column as index(x) (int value)
    fcn = list(fc_month_dict.keys())[list(fc_month_dict.values()).index(month)]
    
    # appends filter value and strips white space
    
    # checks if forecast cycle is 1 (target/budget), need to be kept
    if fc == 1:
        fc_prev_filter.append("Keep")
    
    # checks if FC number is prior to latest FC
    elif fc < forecast_prev:
        
        # check truth of fc number compared to month value (uses dictionary key's to find value)
        if fc_month_dict[fc].strip().lower() == month.strip().lower():
            fc_prev_filter.append("Keep")
        else:
            fc_prev_filter.append("REMOVE")
            
    # checks when FC number is the latest FC
    elif fc == forecast_prev:
    
        # check truth of fc number compared to month value (uses dictionary value to find key)
        # if the month columns value is less than (before) the forecast cycle num, "remove"
        if fcn < fc:
            fc_prev_filter.append("REMOVE")
        else:
            fc_prev_filter.append("Keep")
        
    else:
        fc_prev_filter.append("Fix")
        
        
    x += 1    
    

# Set column name "Value Inclusion" with resulting string from list above    
NPD_PrevFC['Value Inclusion'] = fc_prev_filter

# set backlog value based on latest Forecast cycle -- WORK IN PROGRESS
# v = 0
# for i in NPD_ref['Target Type']:
#     if NPD_ref['Target Type'][v] == "Actual/ Current Forecast" or NPD_ref['Target Type'][v] == "Forcst1":
#         new_value.append(NPD_ref['Value'][v])
#     elif NPD_ref['Target Type'][v] == "Backlog":
#         if NPD_ref['Forecast Number'] == forecast_latest:
#             new_value.append(NPD_ref['Value'][v])
#         else:
#             new_value.append(0)
#     else:
#         continue
#     x +=1


# Filter df's (df of previous fc)
NPD_PrevFC = NPD_PrevFC[NPD_PrevFC['Value Inclusion'] == "Keep"]
NPD_PrevFC = NPD_PrevFC[NPD_PrevFC['Trade Revenue'] == "Vitality"] 


# Remove unnecessary columns from dataframe (these were used to clean and format the final data)
NPD_PrevFC = NPD_PrevFC.drop('Value Inclusion',axis=1)
NPD_PrevFC = NPD_PrevFC.drop('Forecast Number',axis=1)

print("Complete")

Complete


In [2]:
# Generate manipulated file
today = datetime.now()
forecast = forecast_latest

# saves actual's file
file_name_actual = '2023_NPD_Actual.xlsx'
workbook_actual = pd.ExcelWriter(file_name_actual,datetime_format='mm/dd/yy')
NPD_Actual.to_excel(workbook_actual , "Sheet_1", index = False)
workbook_actual.save()

# saves actual's file in ALL NPD directory
file_name_all = '2023_NPD.xlsx'
file_path_all = "C://Users//andrew.schelberg//OneDrive - Vontier//Desktop//Financial Reporting//All NPD//"
file_path_name_all = file_path_all + file_name_all
workbook_actual_all = pd.ExcelWriter(file_path_name_all,datetime_format='mm/dd/yy')
NPD_Actual.to_excel(workbook_actual_all, "Sheet_1", index = False)
workbook_actual_all.save()

#saves prev fv file
file_name_prev = '2023_NPD_PrevFC.xlsx'
workbook_prev = pd.ExcelWriter(file_name_prev,datetime_format='mm/dd/yy')
NPD_PrevFC.to_excel(workbook_prev , "Sheet_1", index = False)
workbook_prev.save()

# saves actual's file in ALL NPD directory
file_name_fc = f'2023_NPD_FC{forecast}.xlsx'
file_path_fc = "C://Users//andrew.schelberg//OneDrive - Vontier//Desktop//Financial Reporting//2023//0 - RevDm//00 - Trial Process//Data Files//Forecast Cycle Files//"
file_path_name_fc = file_path_fc + file_name_fc
workbook_fc = pd.ExcelWriter(file_path_name_fc,datetime_format='mm/dd/yy')
NPD_Actual.to_excel(workbook_fc , "Sheet_1", index = False)
workbook_fc.save()